In [1]:
import numpy as np
import pandas as pd
from __future__ import print_function # Not needed for Kaggle
from sklearn.model_selection import train_test_split # Not for Kaggle

In [2]:
!ls data

sample_submission.csv test.csv              train.csv


In [4]:
trainData = pd.read_csv("data/train.csv") 
trainData, testData = train_test_split(trainData) # shuffle=False resulted in "Invalid parameters passed". Why?
subm = pd.read_csv("data/sample_submission.csv")
# Kaggle:
#trainData = pd.read_csv("../input/train.csv") 
#testData = pd.read_csv("../input/test.csv")
#subm = pd.read_csv('../input/sample_submission.csv')

In [5]:
trainData.dtypes

id               object
comment_text     object
toxic             int64
severe_toxic      int64
obscene           int64
threat            int64
insult            int64
identity_hate     int64
dtype: object

In [6]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
trainData['none'] = 1 - trainData[label_cols].max(axis=1)
trainData.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
count,119678.000000,119678.000000,119678.000000,119678.000000,119678.000000,119678.000000,119678.000000
mean,0.095849,0.010102,0.052792,0.003016,0.049282,0.008849,0.898419
std,0.294385,0.100001,0.223618,0.054839,0.216458,0.093651,0.302098
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
trainData.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
37189,63424c7da983066b,""":You simply refuse to hear other editors and ...",0,0,0,0,0,0,1
63700,aa72fbb41028624c,Its better and cooler in every way!!!!!!!!!!!!...,0,0,0,0,0,0,1
19121,327105ce2d50994e,Falk's own wife and close friend Joe Mantegna ...,0,0,0,0,0,0,1
131685,c090b61ea16f8512,"""\n\n Celine Dion Signings \n\nPlease do not c...",0,0,0,0,0,0,1
145169,183285923f6f8aa6,"""\n\nPresent Day Canada Government website inc...",0,0,0,0,0,0,1


In [8]:
#trainDataLabels = trainData[label_cols] 

In [9]:
#trainDataLabels.dtypes

toxic            int64
severe_toxic     int64
obscene          int64
threat           int64
insult           int64
identity_hate    int64
dtype: object

In [10]:
#trainDataLabels.shape

(159571, 6)

In [11]:
#trainDocs, testDocs, trainLabels, testLabels = train_test_split(trainData.comment_text, trainDataLabels)

In [12]:
#print(trainDocs.shape, testDocs.shape, trainLabels.shape, testLabels.shape)

(119678,) (39893,) (119678, 6) (39893, 6)


In [9]:
trainData[trainData.toxic==1].sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
16006,2a3a48e6463ee509,BECAUSE HE IS A FAGOT,1,0,1,0,0,1,0
102881,269ff30444c96d18,I did'nt vandalize anything. Faggot. 166.248.6...,1,0,1,0,1,0,0
105270,332d75516174c0d1,Leave our country. \n\nWe don't want you Arabs...,1,0,0,0,0,1,0
56279,96652d8e8b2a9b4f,Tread along dear boy \n\nGoway ya fool you th...,1,0,0,0,0,0,0
109186,47e79dce09f476cd,Idiotic Sad Case \n\nYou're an ugly dumb slut ...,1,0,1,0,1,0,0


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer(ngram_range=(1,2)) # ToDo: try min_df=3
# Also: max_df=0.9, strip_accents='unicode', use_idf=1, smooth_idf=1, sublinear_tf=1
# tokenizer=tokenize where tokenize is defined in Jeremy Howard's NB-SVM
trainDocsTfidf = tfidfv.fit_transform(trainData.comment_text)
trainDocsTfidf.shape

(119678, 2004598)

In [29]:
from sklearn.naive_bayes import MultinomialNB
labelsClf = { label: MultinomialNB().fit(trainDocsTfidf, trainData[label]) for label in label_cols}

In [30]:
testdocs = ['Fucking pig', 'Love love love.']
testdocs_tfidf = tfidfv.transform(testdocs)
print(testdocs_tfidf.shape)
for label in label_cols:
    print(label, labelsClf[label].predict(testdocs_tfidf))

(2, 2004598)
toxic [1 0]
severe_toxic [0 0]
obscene [0 0]
threat [0 0]
insult [0 0]
identity_hate [0 0]


In [31]:
# For local test only
testdocs_tfidf = tfidfv.transform(testData.comment_text)
for label in label_cols:
    testPredicts = labelsClf[label].predict(testdocs_tfidf)
    print(label, np.mean(testPredicts==testData[label]))

toxic 0.910686085278
severe_toxic 0.990324117013
obscene 0.948512270323
threat 0.997067154639
insult 0.950642969945
identity_hate 0.991326799188


The accuracy result as executed at 5:50pm 2/15 are shown in the middle column.
The far right column shows the accuracies of Jeremy Howard's [NB-SVM kernel](https://www.kaggle.com/jhoward/nb-svm-strong-linear-baseline/notebook), modified to use 9% of the training data to fit in my computer. 

| label | NB-SVM | my base | 3-gram | 2-gram |
| ----- | ------ | ------- | ------ | ------ |
| toxic | 0.9242 | 0.9197 | \>0.9089 | \>0.9107
| severe_toxic | 0.9881 | 0.9897 | <0.9903 | <0.9903 |
| obscene | 0.9492 | 0.9524 | \>0.9479 | \>0.9485 |
| threat | 0.9944 | 0.9971 | =0.9971 | ~=0.9970 |
| insult | 0.9518 | 0.9527 | \>0.9506 | \>0.9506 |
| identity_hate | 0.9893 | 0.9914 | ~=0.9913 | ~=0.9913 |

In [16]:
len(testData)

39893

In [22]:
# For Kaggle
preds = np.zeros((len(testData), len(label_cols)))
testdocs_tfidf = tfidfv.transform(testData.comment_text)
for i, label in enumerate(label_cols):
    preds[:,i] = labelsClf[label].predict_proba(testdocs_tfidf)[:,1]
#subm = pd.read_csv("data/sample_submission.csv")
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv('submission.csv', index=False)

In [20]:
preds.shape

(39893, 6)